## **Playing TicTacToe with Reinforcement Learning & OpenAI Gym**

In [1]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/playing-tictactoe-with-reinforcement-learning-and-openai-gym/gym-tictactoe.zip

--2022-07-09 09:09:12--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/playing-tictactoe-with-reinforcement-learning-and-openai-gym/gym-tictactoe.zip
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10119 (9.9K) [application/zip]
Saving to: ‘gym-tictactoe.zip’

gym-tictactoe.zip   100%[===================>]   9.88K  --.-KB/s    in 0s      

2022-07-09 09:09:14 (91.2 MB/s) - ‘gym-tictactoe.zip’ saved [10119/10119]



In [2]:
!unzip -o gym-tictactoe.zip

Archive:  gym-tictactoe.zip
   creating: gym-tictactoe/
   creating: gym-tictactoe/gym_tictactoe.egg-info/
   creating: gym-tictactoe/gym_tictactoe/
  inflating: gym-tictactoe/setup.py  
   creating: gym-tictactoe/.ipynb_checkpoints/
  inflating: gym-tictactoe/gym_tictactoe.egg-info/PKG-INFO  
  inflating: gym-tictactoe/gym_tictactoe.egg-info/SOURCES.txt  
  inflating: gym-tictactoe/gym_tictactoe.egg-info/requires.txt  
  inflating: gym-tictactoe/gym_tictactoe.egg-info/top_level.txt  
  inflating: gym-tictactoe/gym_tictactoe.egg-info/dependency_links.txt  
  inflating: gym-tictactoe/gym_tictactoe/__init__.py  
   creating: gym-tictactoe/gym_tictactoe/__pycache__/
   creating: gym-tictactoe/gym_tictactoe/.ipynb_checkpoints/
   creating: gym-tictactoe/gym_tictactoe/envs/
  inflating: gym-tictactoe/.ipynb_checkpoints/setup-checkpoint.py  
  inflating: gym-tictactoe/gym_tictactoe/__pycache__/__init__.cpython-39.pyc  
  inflating: gym-tictactoe/gym_tictactoe/.ipynb_checkpoints/__init__-chec

In [1]:
import gym
import random
import gym_tictactoe

In [2]:
env = gym.make("TicTacToe-v0")

In [3]:
env.state

[['-', '-', '-'], ['-', '-', '-'], ['-', '-', '-']]

In [4]:
env.hash()

'---------'

In [5]:
new_state, reward, done, info = env.step(0, "X")

In [6]:
new_state

'X--------'

In [7]:
reward

(0, 0)

In [8]:
done

False

In [9]:
info

{}

In [10]:
env.render()

Board
['X', '-', '-']
['-', '-', '-']
['-', '-', '-']


In [11]:
env.available_actions()

[1, 2, 3, 4, 5, 6, 7, 8]

In [12]:
env.available_states("O")

[('XO-------', (0, 0)),
 ('X-O------', (0, 0)),
 ('X--O-----', (0, 0)),
 ('X---O----', (0, 0)),
 ('X----O---', (0, 0)),
 ('X-----O--', (0, 0)),
 ('X------O-', (0, 0)),
 ('X-------O', (0, 0))]

In [13]:
env.check_done(env.hash())

(False, (0, 0))

In [14]:
env.reset()
env.render()

Board
['-', '-', '-']
['-', '-', '-']
['-', '-', '-']


In [15]:
done = False
env.reset()
env.render()
while not done:
    new_state, reward, done, info = env.step(
        random.choice(env.available_actions()), "X")
    env.render()

    if not done:
        new_state, reward, done, info = env.step(
            random.choice(env.available_actions()), "O")
        env.render()

print(reward)

Board
['-', '-', '-']
['-', '-', '-']
['-', '-', '-']
Board
['-', '-', '-']
['-', '-', '-']
['X', '-', '-']
Board
['-', '-', '-']
['-', '-', 'O']
['X', '-', '-']
Board
['-', '-', '-']
['-', '-', 'O']
['X', '-', 'X']
Board
['-', '-', '-']
['O', '-', 'O']
['X', '-', 'X']
Board
['X', '-', '-']
['O', '-', 'O']
['X', '-', 'X']
Board
['X', 'O', '-']
['O', '-', 'O']
['X', '-', 'X']
Board
['X', 'O', '-']
['O', 'X', 'O']
['X', '-', 'X']
(10, -10)


In [16]:
done = False
env.reset()
while not done:
    new_state, reward, done, info = env.step(
        random.choice(env.available_actions()), "X")
    print(env.hash())

    if not done:
        new_state, reward, done, info = env.step(
            random.choice(env.available_actions()), "O")
        print(env.hash())

---X-----
---X--O--
X--X--O--
X-OX--O--
X-OX--O-X
X-OX--OOX
XXOX--OOX
XXOXO-OOX


In [17]:
env.reset()
env.render()

Board
['-', '-', '-']
['-', '-', '-']
['-', '-', '-']


In [18]:
reward

(-10, 10)

In [19]:
env.available_actions()

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [20]:
class Agent():
    def __init__(self, env, player="X", alpha=0.4, gamma=0.9):
        self.alpha = alpha
        self.gamma = gamma
        self.env = env
        self.player = player
        self.player_number = 0 if player == "X" else 1
        self.V = {}

In [21]:
class Agent(Agent):
    def select_action(self, epsilon=0.1):
        if (random.random() < epsilon):
            action = random.choice(self.env.available_actions())
        else:
            q_values = []
            for state in self.env.available_states(self.player):
                q_values.append(self.gamma*self.V[state[0]] + state[1][self.player_number])
            
            max_value = max(q_values)
            max_indexs = [i for i, j in enumerate(q_values) if j == max_value]
            action = self.env.available_actions()[random.choice(max_indexs)]
        
        return action

In [22]:
class Agent(Agent):
    def add_states(self):
        if (self.env.hash() not in self.V):
            self.V[self.env.hash()] = 0
        for state in self.env.available_states("X"):
            if (state[0] not in self.V):
                self.V[state[0]] = 0

        for state in self.env.available_states("O"):
            if (state[0] not in self.V):
                self.V[state[0]] = 0

In [23]:
class Agent(Agent):
    def update_state_values(self, new_state, old_state, reward):
        self.V[old_state] = self.V[old_state] + self.alpha*(reward + self.gamma*self.V[new_state] - self.V[old_state])

In [24]:
def train(episodes):
    agent_x = Agent(env, "X")
    agent_o = Agent(env, "O")
    for episode in range(episodes):
        done = False
        env.reset()
        while not done:
            agent_x.add_states()
            agent_o.add_states()
            old_state = env.hash()
            action = agent_x.select_action()
            new_state, reward, done, _ = env.step(action, agent_x.player)
            agent_x.update_state_values(new_state, old_state, reward[agent_x.player_number])
            agent_o.update_state_values(new_state, old_state, reward[agent_o.player_number])
            if not done:
                agent_x.add_states()
                agent_o.add_states()
                old_state = env.hash()
                action = agent_o.select_action()
                new_state, reward, done, _ = env.step(action, agent_o.player)
                agent_x.update_state_values(new_state, old_state, reward[agent_x.player_number])
                agent_o.update_state_values(new_state, old_state, reward[agent_o.player_number])

    return agent_x, agent_o

In [25]:
%%time

agent_x, agent_o = train(110000)

CPU times: user 1min 31s, sys: 58.1 ms, total: 1min 31s
Wall time: 1min 31s


In [26]:
def test_x(episodes):
    win = 0
    tie = 0
    loss = 0
    for episode in range(episodes):
        done = False
        env.reset()
        while not done:
            agent_x.add_states()
            x_action = agent_x.select_action(epsilon=0)
            new_state, reward, done, _ = env.step(x_action, agent_x.player)
            if (not done):
                agent_x.add_states()
                o_action = random.choice(env.available_actions())
                new_state, reward, done, _ = env.step(o_action, "O")
        
        if (reward == (10, -10)):
            win += 1
        elif (reward == (-10, 10)):
            loss += 1
        elif (reward == (0, 0)):
            tie += 1
    return win, loss, tie

In [27]:
episodes = 10000
win, loss, tie = test_x(episodes)

print("Win:", win, "Tie:", tie, "Loss:", loss)
print("Win Rate:", win/episodes*100, "Tie Rate:", tie / episodes*100, "Loss Rate:", loss/episodes*100)

Win: 9902 Tie: 98 Loss: 0
Win Rate: 99.02 Tie Rate: 0.98 Loss Rate: 0.0


In [28]:
def test_o(episodes):
    win = 0
    tie = 0
    loss = 0
    for episode in range(episodes):
        done = False
        env.reset()
        while not done:
            agent_o.add_states()
            x_action = random.choice(env.available_actions())
            new_state, reward, done, _ = env.step(x_action, "X")
            if (not done):
                agent_o.add_states()
                o_action = agent_o.select_action(epsilon=0)
                new_state, reward, done, _ = env.step(o_action, agent_o.player)

        if (reward == (-10, 10)):
            win += 1
        elif (reward == (10, -10)):
            loss += 1
        elif (reward == (0, 0)):
            tie += 1
    return win, loss, tie

In [29]:
episodes = 10000
win, loss, tie = test_o(episodes)

print("Win:", win, "Tie:", tie, "Loss:", loss)
print("Win Rate:", win/episodes*100, "Tie Rate:", tie / episodes*100, "Loss Rate:", loss/episodes*100)

Win: 8520 Tie: 1473 Loss: 7
Win Rate: 85.2 Tie Rate: 14.729999999999999 Loss Rate: 0.06999999999999999


In [30]:
def test(episodes):
    x_win = 0
    o_win = 0
    tie = 0
    for episode in range(episodes):
        done = False
        env.reset()
        while not done:
            agent_x.add_states()
            agent_o.add_states()
            x_action = agent_x.select_action(epsilon=0)
            new_state, reward, done, _ = env.step(x_action, "X")
            if (not done):
                agent_x.add_states()
                agent_o.add_states()
                o_action = agent_o.select_action(epsilon=0)
                new_state, reward, done, _ = env.step(o_action, "O")

        if (reward == (-10, 10)):
            o_win += 1
        elif (reward == (10, -10)):
            x_win += 1
        elif (reward == (0, 0)):
            tie += 1
    return x_win, o_win, tie

In [31]:
episodes = 10000
x_win, o_win, tie = test(episodes)

print("X Win:", x_win, "Tie:", tie, "O Win:", o_win)
print("X Win Rate:", x_win/episodes*100, "Tie Rate:", tie/episodes*100, "O Win Rate:", o_win/episodes*100)

X Win: 0 Tie: 10000 O Win: 0
X Win Rate: 0.0 Tie Rate: 100.0 O Win Rate: 0.0


In [32]:
def play_as_x(episodes=1):
    x_win = 0
    o_win = 0
    tie = 0
    for episode in range(episodes):
        done = False
        env.reset()
        while not done:
            env.render()
            print(env.available_actions())
            agent_o.add_states()
            x_action = int(input())
            new_state, reward, done, _ = env.step(x_action, "X")
            if (not done):
                agent_o.add_states()
                o_action = agent_o.select_action(epsilon=0)
                new_state, reward, done, _ = env.step(o_action, "O")

        env.render()
        if (reward == (-10, 10)):
            print("You Lose")
        elif (reward == (10, -10)):
            print("You Win")
        elif (reward == (0, 0)):
            print("Tie")

In [34]:
play_as_x()

Board
['-', '-', '-']
['-', '-', '-']
['-', '-', '-']
[0, 1, 2, 3, 4, 5, 6, 7, 8]
Board
['-', 'X', '-']
['-', 'O', '-']
['-', '-', '-']
[0, 2, 3, 5, 6, 7, 8]
Board
['O', 'X', 'X']
['-', 'O', '-']
['-', '-', '-']
[3, 5, 6, 7, 8]
Board
['O', 'X', 'X']
['X', 'O', '-']
['-', '-', 'O']
You Lose


In [35]:
def play_as_o(episodes=1):
    x_win = 0
    o_win = 0
    tie = 0
    for episode in range(episodes):
        done = False
        env.reset()
        while not done:
            agent_x.add_states()
            x_action = agent_x.select_action(epsilon=0)
            new_state, reward, done, _ = env.step(x_action, "X")
            if (not done):
                env.render()
                print(env.available_actions())
                agent_x.add_states()
                o_action = int(input())
                new_state, reward, done, _ = env.step(o_action, "O")

        env.render()
        if (reward == (-10, 10)):
            print("You Win")
        elif (reward == (10, -10)):
            print("You Lose")
        elif (reward == (0, 0)):
            print("Tie")

In [36]:
play_as_o()

Board
['-', '-', '-']
['-', 'X', '-']
['-', '-', '-']
[0, 1, 2, 3, 5, 6, 7, 8]
Board
['O', 'X', '-']
['-', 'X', '-']
['-', '-', '-']
[2, 3, 5, 6, 7, 8]
Board
['O', 'X', '-']
['O', 'X', '-']
['-', 'X', '-']
You Lose


In [37]:
def train_o_against_random(episodes):
    agent_o = Agent(env, "O")
    for episode in range(episodes):
        done = False
        env.reset()
        while not done:
            agent_o.add_states()
            old_state = env.hash()
            action = random.choice(env.available_actions())
            new_state, reward, done, _ = env.step(action, "X")
            agent_o.update_state_values(new_state, old_state, reward[agent_o.player_number])
            if not done:
                agent_o.add_states()
                old_state = env.hash()
                action = agent_o.select_action()
                new_state, reward, done, _ = env.step(action, agent_o.player)
                agent_o.update_state_values(new_state, old_state, reward[agent_o.player_number])

    return agent_o

In [38]:
agent_o = train_o_against_random(110000)

In [39]:
episodes = 10000
win, loss, tie = test_o(episodes)

print("Win:", win, "Tie:", tie, "Loss:", loss)
print("Win Rate:", win/episodes*100, "Tie Rate:", tie / episodes*100, "Loss Rate:", loss/episodes*100)

Win: 9094 Tie: 731 Loss: 175
Win Rate: 90.94 Tie Rate: 7.31 Loss Rate: 1.7500000000000002
